In [1]:
#2
#athena db config - aimodelpoll
#AAI-540 Group 3 FP

In [3]:
import boto3
import sagemaker
from pyathena import connect
import pandas as pd
sess = sagemaker.Session()
bucket = sess.default_bucket()
role = sagemaker.get_execution_role()
region = boto3.Session().region_name
ingest_create_athena_db_passed = False

In [4]:
database_name = "dsoaws"
s3_staging_dir = "s3://{0}/athena/staging".format(bucket)
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

In [5]:
statement = "CREATE DATABASE IF NOT EXISTS {}".format(database_name)
print(statement)
pd.read_sql(statement, conn)

CREATE DATABASE IF NOT EXISTS dsoaws


/tmp/ipykernel_4059/2532319348.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [6]:
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_4059/1677879827.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws
2,sagemaker_featurestore


In [22]:
s3_data_path = 's3://sagemaker-us-east-1-883727203134/aimodelpoll.csv'
s3_table_path = 's3://sagemaker-us-east-1-883727203134/table1/'

In [23]:
!aws s3 cp {s3_data_path} {s3_table_path}

copy: s3://sagemaker-us-east-1-883727203134/aimodelpoll.csv to s3://sagemaker-us-east-1-883727203134/table1/aimodelpoll.csv


In [25]:
#table1 - aimodelpoll.csv
table_name_csv = 'aimodelpoll'
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)
ingest_create_athena_table_csv_passed = False
dataexplore = pd.read_csv('aimodelpoll.csv')
dataexplore.head()

,Question,Response,Overall,Age: 18-34,Age: 35-49,Age: 50-64,Age: 65+,Gender: Women,Gender: Men,College/No College: College,...,Race: Native American,Race: Asian,Race: Hispanic,Race: Black,Race: White,Occupation Category: Office Job,Occupation Category: Customer Facing Service Job,"Occupation Category: Manual, Not Customer Facing",Occupation Category: Education/Medicine,Occupation Category: Building Trades
0,AI Services Used,ChatGPT,34.8%,48.8%,40.7%,32.8%,17.1%,34.1%,35.5%,41.4%,...,33.0%,40.2%,37.5%,35.2%,33.7%,39.9%,34.2%,29.9%,36.0%,31.7%
1,AI Services Used,Claude,1.7%,3.1%,1.8%,1.2%,0.6%,1.6%,1.8%,1.7%,...,1.9%,2.2%,2.4%,2.4%,1.4%,2.2%,2.3%,1.9%,2.2%,2.0%
2,AI Services Used,Gemini,24.3%,27.9%,29.9%,23.8%,16.0%,22.5%,26.3%,25.4%,...,25.5%,28.0%,27.6%,30.4%,22.1%,28.6%,27.9%,28.1%,27.6%,28.5%
3,AI Services Used,Grok,3.8%,6.3%,4.3%,2.9%,1.8%,2.8%,4.9%,3.5%,...,3.8%,4.3%,4.7%,3.9%,3.6%,4.3%,4.0%,4.4%,3.8%,4.8%
4,AI Services Used,Copilot,12.2%,11.5%,14.0%,14.5%,9.2%,10.8%,13.7%,16.6%,...,11.2%,15.2%,11.5%,12.9%,11.9%,14.9%,10.2%,10.9%,13.4%,12.1%


In [26]:
dataexplore.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39 entries, 0 to 38
Data columns (total 29 columns):
 #   Column                                            Non-Null Count  Dtype 
---  ------                                            --------------  ----- 
 0   Question                                          39 non-null     object
 1   Response                                          37 non-null     object
 2   Overall                                           39 non-null     object
 3   Age: 18-34                                        39 non-null     object
 4   Age: 35-49                                        39 non-null     object
 5   Age: 50-64                                        39 non-null     object
 6   Age: 65+                                          39 non-null     object
 7   Gender: Women                                     39 non-null     object
 8   Gender: Men                                       39 non-null     object
 9   College/No College: College       

In [27]:
!aws s3 cp s3://sagemaker-us-east-1-883727203134/aimodelpoll.csv \
           s3://sagemaker-us-east-1-883727203134/table1/

copy: s3://sagemaker-us-east-1-883727203134/aimodelpoll.csv to s3://sagemaker-us-east-1-883727203134/table1/aimodelpoll.csv


In [28]:
!aws s3 ls s3://sagemaker-us-east-1-883727203134/table1/

2026-01-27 03:14:34       7683 aimodelpoll.csv


In [31]:
#sql table creation
statement = """CREATE EXTERNAL TABLE IF NOT EXISTS {}.{}(
         Question STRING,
         Response STRING,
         Overall STRING,
         Age_18_34 STRING,
         Age_35_49 STRING,
         Age_50_64 STRING,
         Age_65_up STRING,
         Gender_Women STRING,
         Gender_Men STRING,
         College_No_College_College STRING,
         College_No_College_No_college STRING,
         Income_three_way_Lower_Income STRING,
         Income_three_way_Middle_Income STRING,
         Income_three_way_Upper_Income STRING,
         Generation_Gen_Z STRING,
         Generation_Millenial STRING,
         Generation_Gen_X STRING,
         Generation_Boomer STRING,
         Generation_Silent STRING,
         Race_Native_American STRING,
         Race_Asian STRING,
         Race_Hispanic STRING,
         Race_Black STRING,
         Race_White STRING,
         Occupation_Category_Office_Job STRING,
         Occupation_Category_Customer_Facing_Service_Job STRING,
         Occupation_Category_Manual_Not_Customer_Facing STRING,
         Occupation_Category_Education_Medicine STRING,
         Occupation_Category_Building_Trades STRING
) ROW FORMAT DELIMITED 
  FIELDS TERMINATED BY ',' 
  LINES TERMINATED BY '\\n' 
LOCATION '{}'
TBLPROPERTIES ('skip.header.line.count'='1')""".format(
    database_name, table_name_csv, s3_table_path
)

In [32]:
pd.read_sql(statement, conn)

/tmp/ipykernel_4059/618921652.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


""


In [33]:
statement = "SHOW TABLES in {}".format(database_name)
df_show = pd.read_sql(statement, conn)
df_show.head(5)

/tmp/ipykernel_4059/84333453.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,tab_name
0,aimodelpoll
1,dataset_csv2
2,llmdatasetcsv


In [34]:
statement = """SELECT * FROM {}.{} LIMIT 10""".format(
    database_name, table_name_csv
)
print(statement)
df = pd.read_sql(statement, conn)
df.head()

SELECT * FROM dsoaws.aimodelpoll LIMIT 10


/tmp/ipykernel_4059/888292236.py:5: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql(statement, conn)


,question,response,overall,age_18_34,age_35_49,age_50_64,age_65_up,gender_women,gender_men,college_no_college_college,...,race_native_american,race_asian,race_hispanic,race_black,race_white,occupation_category_office_job,occupation_category_customer_facing_service_job,occupation_category_manual_not_customer_facing,occupation_category_education_medicine,occupation_category_building_trades
0,AI Services Used,ChatGPT,34.8%,48.8%,40.7%,32.8%,17.1%,34.1%,35.5%,41.4%,...,33.0%,40.2%,37.5%,35.2%,33.7%,39.9%,34.2%,29.9%,36.0%,31.7%
1,AI Services Used,Claude,1.7%,3.1%,1.8%,1.2%,0.6%,1.6%,1.8%,1.7%,...,1.9%,2.2%,2.4%,2.4%,1.4%,2.2%,2.3%,1.9%,2.2%,2.0%
2,AI Services Used,Gemini,24.3%,27.9%,29.9%,23.8%,16.0%,22.5%,26.3%,25.4%,...,25.5%,28.0%,27.6%,30.4%,22.1%,28.6%,27.9%,28.1%,27.6%,28.5%
3,AI Services Used,Grok,3.8%,6.3%,4.3%,2.9%,1.8%,2.8%,4.9%,3.5%,...,3.8%,4.3%,4.7%,3.9%,3.6%,4.3%,4.0%,4.4%,3.8%,4.8%
4,AI Services Used,Copilot,12.2%,11.5%,14.0%,14.5%,9.2%,10.8%,13.7%,16.6%,...,11.2%,15.2%,11.5%,12.9%,11.9%,14.9%,10.2%,10.9%,13.4%,12.1%
